In [1]:
import pandas as pd
import math
import numpy as np
from math import radians, cos, sin, asin, sqrt
import sys
from queue import PriorityQueue
import sys
from math import tanh
import heapq
import numpy as np

In [2]:
def get_data():
    data = pd.read_csv('city-gps.txt', sep = ' ', index_col = None, names=["City", "latitude", "Longitude"])
    lat_long = data.set_index('City').T.to_dict('list')
    data1 = pd.read_csv('road-segments.txt', sep = ' ', index_col = None, names = [ 'first_city', 'second_city', 'length', 'speed_limit', 'name_of_highway'])
    data2 = data1.copy()
    data2.rename(columns={"first_city": "second_city", "second_city": "first_city"},inplace = True )
    data_final = data1.append(data2)
    data_final.drop_duplicates(inplace=True)
    data_list = data_final.values.tolist()
    return lat_long, data_final

In [9]:
lat_long, data_list = get_data()

C:\Users\sarad\AppData\Local\Temp\ipykernel_26040\1146456031.py:3: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  lat_long = data.set_index('City').T.to_dict('list')
C:\Users\sarad\AppData\Local\Temp\ipykernel_26040\1146456031.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_final = data1.append(data2)


In [14]:
def haversine_dist(city1, city2,data):
    if city1 not in data.keys() or city2 not in data.keys():
        return 0
    city1 = data[city1]
    city2 = data[city2]
    # this function gives the haversine distance in miles between any two cities
    city1_long = radians(city1[1])
    city2_long = radians(city2[1])
    city1_lat = radians(city1[0])
    city2_lat = radians(city2[0])

    temp_distance = 2 * asin(sqrt(sin((city2_lat - city1_lat)/2) ** 2 + cos(city1_lat) * cos(city2_lat) * sin((city2_long - city1_long)/2) ** 2))

    return temp_distance*3956


In [4]:
def successors(data_list,city):
    succ = []
    for i in data_list:
        if i[0] == city:
            succ.append(i[1])
    return succ

In [5]:
def is_goal(state,city2):
    return state == city2

In [6]:
def solve(city1,city2, data_list,lat_long):
    fringe = PriorityQueue()
    count = 0
    fringe.put((count,(city1,[])))
    while ( not fringe.empty()):
        
        gg = (fringe.get())
        (state, path) = gg[1]
        
        
        if is_goal(state,city2):
            return path+[state,]
        
        count+=1
        for s in successors(data_list,state):
            fringe.put((len(path)+haversine_dist(s,city2,lat_long),(s, path+[state])))
            
    return []

In [ ]:
def find_max_speed(file):
    city1, city2, distance, speed_limit, highway = np.genfromtxt(file, unpack=True)
    return max(speed_limit)

# Segment cost function
def segment_cost_function(end, current_state, city_gps, road_segments, max_speed):
    current_route, route_taken, current_segment_count, current_miles, current_time, current_delivery_time = current_state
    return current_segment_count + 1


# Distance cost function
def dist_cost_function(end, current_state, city_gps, road_segments, max_speed):
    current_route, route_taken, current_segment_count, current_miles, current_time, current_delivery_time = current_state
    current_city = current_route[-1]
    distance = cities_distance(current_city, end, city_gps)
    return current_miles + distance


# Time cost function
def time_cost_function(end, current_state, city_gps, road_segments, max_speed):
    current_route, route_taken, current_segment_count, current_miles, current_time, current_delivery_time = current_state
    current_city = current_route[-1]
    distance = cities_distance(current_city, end, city_gps)

    time = distance / max_speed
    return current_time + time


def delivery_cost_function(end, current_state, city_gps, road_segments, max_speed):
    current_route, route_taken, current_segment_count, current_miles, current_time, current_delivery_time = current_state
    current_city = current_route[-1]
    distance = cities_distance(current_city, end, city_gps)

    time = distance / max_speed
    return current_delivery_time + time



# Select the proper cost function
def cost_function_selection(cost_func, end, current_state, city_gps, road_segments, max_speed):
    if cost_func == 'segments':
        return segment_cost_function(end, current_state, city_gps, road_segments, max_speed)
    if cost_func == 'distance':
        return dist_cost_function(end, current_state, city_gps, road_segments, max_speed)
    if cost_func == 'time':
        return time_cost_function(end, current_state, city_gps, road_segments, max_speed)
    else:
        return delivery_cost_function(end, current_state, city_gps, road_segments, max_speed)

In [30]:
def find_max_speed(file):
    city1, city2, distance, speed_limit, highway = np.genfromtxt(file, unpack=True)
    return max(speed_limit)

In [34]:
max_speed = find_max_speed('road-segments.txt')

In [35]:
max_speed

nan

In [33]:
def find_max_speed(file):
    city1, city2, distance, speed_limit, highway = np.genfromtxt(file, unpack=True)
    return max(city1)